# Variables
### What’s wrong with constants?
* Constants are stored in the graph definition, This makes loading graphs expensive when constants are big
* Only use constants for primitive types. Use variables or readers for more data that requires more memory
*  A variable is stored separately, and may live on a parameter server

In [1]:
import tensorflow as tf
my_const = tf.constant([10.0, 4.0], name="my_const")
with tf.Session() as sess:
    print(sess.graph.as_graph_def())
# you will see value of my_const stored in the graph’s definition

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


node {
  name: "my_const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000 A\000\000\200@"
      }
    }
  }
}
versions {
  producer: 24
}



In [1]:
import tensorflow as tf
# # created this function to evaluate the expression and print the result.
# like should not be done in production
def run_print(x):
    with tf.Session() as sess:
        print(sess.run(x))



C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Declare variables
To declare a variable, you create an instance of the class tf.Variable. 
**Note** that it’s tf.constant but tf.Variable and not tf.variable because tf.constant is an op, while tf.Variable is a class.

In [6]:
# create a variable with scalar value
a = tf.Variable(2, name="scalar")

# create a variable b as a vector 
b = tf.Variable([2,3],name="vector")

#create a variable as 2X2 matrix
c = tf.Variable([[0,1],[2,3]], name ="matrix")

#create variable W as 784 X 10 tensor, filled with zeros
W = tf.Variable(tf.zeros([784,10]))

#### tf.Variable holds several operations:

> X = tf.Variable(...)

> x.initializer # init

> x.value() # read op

> x.assign(...) # write op

> x.assign_add(...)

#### and more

** You have to initialize variables before using them. ** If you ​ try to evaluate the variables 
before initializing them you’ll run into FailedPreconditionError: Attempting to use uninitialized value tensor.

The easiest way is initializing all variables at once using: tf.global_variables_initializer()

In [11]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    print(sess.run(init))

None


** Note that you use sess.run() to run the initializer, not fetching any value.**

To initialize only a subset of variables, you use tf.variables_initializer() with a list of variables you want to initialize:

In [13]:
init_ab = tf.variables_initializer([a, b], name="init_ab")
with tf.Session() as sess:
    print(sess.run(init_ab))

None


You can also initialize each variable separately using tf.Variable.initializer

In [15]:
# create variable W as 784 x 10 tensor, filled with zeros
W = tf.Variable(tf.zeros([784,10]))
with tf.Session() as sess:
    print(sess.run(W.initializer))

None


Another way to initialize a variable is to restore it from a save file. 

### Evaluate values of variables
If we print the initialize variable, we only see tensor object

In [17]:
W = tf.Variable(tf.truncated_normal([700,10]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W)

<tf.Variable 'Variable_5:0' shape=(700, 10) dtype=float32_ref>


To get the value of a variable, we need to evaluate it using eval()

In [22]:
W = tf.Variable(tf.truncated_normal([700,10]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W.eval())

[[-0.6610072   1.8762355   0.5948747  ...  0.5671912   1.0676944
  -0.31387746]
 [ 0.5552418  -0.94517386  1.0100793  ...  0.855455   -1.960183
   0.80169415]
 [-0.17341878 -0.99093884 -1.0161394  ...  0.93140334 -0.26927486
   0.088343  ]
 ...
 [ 0.00364401  0.78493756  0.25180912 ...  0.71139073 -0.3893471
   0.503082  ]
 [-0.01952268 -1.4283446   0.37384334 ... -1.9144452  -0.7070146
   0.5691087 ]
 [-0.7378621  -1.9790567  -1.2974881  ... -0.03594809  0.6627822
  -1.4918966 ]]


### Assign values to variables
We can assign a value to a variable using tf.Variable.assign()

In [23]:
W = tf.Variable(10)
W.assign(100)
with tf.Session() as session:
    session.run(W.initializer)
    print(W.eval())

10


** Uh, why? ** W.assign(100) doesn’t assign the value 100 to W. It creates an assign op, and that op needs to be run to take effect.

In [24]:
W = tf.Variable(10)
assign_op = W.assign(100)
with tf.Session() as session:
    session.run(assign_op)
    print(W.eval())

100


* You don’t need to initialize variable because assign_op does it for you.

* In fact, initializer op is the assign op that assigns the variable’s initial value to the variable itself.

### Interesting example:

In [25]:
# create a variable whose original value is 2
a = tf.Variable(2, name="scalar")

# assign a * 2 to a and call that op a_times_two
a_times_two = a.assign(a * 2)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    # Have to inintialize a, becoz a_times_two op depends on the value of a
    print(sess.run(a_times_two))
    print(sess.run(a_times_two))
    print(sess.run(a_times_two))

4
8
16


TensorFlow assigns a*2 to a every time `a_times_two` is fetched.

For simple incrementing and decrementing of variables, TensorFlow includes the ` tf.Variable.assign_add()` and `tf.Variable.assign_sub()` methods. Unlike `tf.Variable.assign()`, `tf.Variable.assign_add()` and `tf.Variable.assign_sub()` don’t initialize your variables for you because these ops depend on the initial values of the variable.

In [26]:
W = tf.Variable(10)

with tf.Session() as sess:
    sess.run(W.initializer)
    print(sess.run(W.assign_add(2)))
    print(sess.run(W.assign_sub(4)))

12
8


Because TensorFlow sessions maintain values separately, each Session can have its own current
value for a variable defined in a graph.

In [27]:
W = tf.Variable(10)

session1 = tf.Session()
session2 = tf.Session()

session1.run(W.initializer)
session1.run(W.initializer)

print(session1.run(W.assign_add(10)))
print(session1.run(W.assign_sub(2)))

print(session1.run(W.assign_add(100)))
print(session1.run(W.assign_sub(20)))


session1.close()
session2.close()

20
18
118
98


#### Use a variable to initialize another variable
Want to declare U = 2 * W


In [28]:
W = tf.Variable(tf.truncated_normal([700,10]))
U = tf.Variable(2 * W)

#not safe (but quite common)

In [32]:
W = tf.Variable(tf.truncated_normal([700,10]))
U = tf.Variable(2 * W.initialized_value())

# ensure that W is initialized before its value is used to initialize U -> Safer